In [ ]:


import pandas as pd
import numpy as np
from pandas import DataFrame
import os
os.chdir('G:\\Banglalink\\Python Master\\PythonForModellers-master\\Data')

In [ ]:
data=pd.read_csv('terror.csv',sep=',',header=0, encoding="latin")

print (data.head())
print (data.shape)
print (data.columns.tolist())


In [ ]:

### How many attacks happened in India
data['country_txt'].unique().tolist()
data[data['country_txt']=="India"]
data[data['country_txt']=="India"].shape[0]


In [ ]:
## How many attacks happened in India and upto 3 people were killed
data[(data['country_txt']=='India')&(data['nkill']<=3)].shape[0]


In [ ]:
# This looks ugly
data.query("country_txt=='India' & nkill<=3").shape[0]


In [ ]:
# Extract the city and summary for attacks above
data.query("country_txt=='India' & nkill<=3")[['city','summary']].head(10)


In [ ]:
# In a single terror incident in India, find out top 5 cities by number killed
data.query("country_txt=='India'").sort_values('nkill',ascending=False)['city'].head(5)


In [ ]:
# Little more detailed
data.query("country_txt=='India'").sort_values('nkill',ascending=False)[['city','nkill','iyear']].head(5)


In [ ]:

# In a single terror incident in India, find out top 5 cities by number killed and wounded
data.query("country_txt=='India'").sort_values(['nkill','nwound'],ascending=[False,False])['city'].head(5)


In [ ]:
#### Adding new columns to the data
#Attacks which were successful and were suicide attacks
data['success_suicide']=data['suicide']+data['success']
data=data.assign(success_suicide_1=data['suicide']+data['success'])

data.query("success_suicide==2").shape[0]


In [ ]:

### Dropping columns
data.drop('success_suicide',axis=1) 
data.drop('success_suicide',axis=1,inplace=True) 


In [ ]:

### Aggregations and manipulations using apply and map
#map: map a function to each element of a series object
data['nkill'].describe()
#Suppose we want to label all the incidents where the number killed was more than 5 as severe. This would involve applying a function on each element of the series, map helps in doing that
def get_label(x):
    if x>5:
        return 'Severe'
    else:
        return 'Not Severe'
data['nkill'].map(get_label)
#You can use lambda functions as well (if else follows a special form when used with lambdas )
data['nkill'].map(lambda x: "Severe" if x>5 else "Not Severe")

In [ ]:
# We can use apply to use a function row wise or column wise
# Let's write a function to label an incident that was both successful and suicidal

def get_label(row):
    if row['success']==1 and row['suicide']==1:
        return 1
    else:
        return 0
data.apply(get_label,axis=1)
data.apply(get_label,axis=1).unique()

data.apply(lambda row: 1 if row['success']==1 and row['suicide']==1 else 0,axis=1)


In [ ]:

### Create a new  category representing if the incident occured in Afghanistan, Pakistan or India as one level of the category and all the other countries as another level

def get_label(row):
    if row['country_txt']=='India' or row['country_txt']=='Afghnistan' or row['country_txt']=='Pakistan':
        return 'Af-Pak-India'
    else:
        return 'ROW'
data.apply(get_label,axis=1)

data['Local']=data.apply(get_label,axis=1)


In [ ]:
data.apply(lambda row: 'Af-Pak-India' if row['country_txt']=='India' or row['country_txt']=='Afghnistan' or row['country_txt']=='Pakistan' else  'ROW', axis =1 )

In [ ]:

## Number of incidents in Af-Pak-India vs ROW
data['Local'].value_counts()
data.groupby('Local').size()

In [ ]:


## Number of suicide attacks by Af-Pak-India vs ROW
data.groupby(['Local','suicide']).agg({'eventid':np.size})
data.groupby(['Local','suicide'],as_index=False).agg({'eventid':np.size})


In [ ]:

## Number of suicides attacks and average kills by Af-Pak-India vs ROW
data.groupby(['Local','suicide'])['nkill'].agg([np.mean,np.size])
data.groupby(['Local','suicide'])['nkill'].agg([np.mean,np.size]).reset_index()

In [ ]:
#Rename columns
data.groupby(['Local','suicide'])['nkill'].agg([np.mean,np.size]).reset_index().rename(columns={'mean':'Average_Kills','size':'Number_Incidents'})

data.groupby(["Local",'suicide']).agg({'nkill':np.mean,'nwound':np.sum}).reset_index().rename(columns={'nwound':'Total_Wounded','nkill':'Number_Killed'})
